In [1]:
import pandas as pd
from customrec_engine import engines_list, engines, RecommendationAbstract
import pprint
from sklearn.model_selection import train_test_split

product_datas = [{
    "data_context": "books",
    "product_filepath": "data/products_books_v1_10_10.csv",
    "transactions_filepath": "data/transactions_books_v1_10_10.csv",
    "features": ["product_title", "product_image", "product_soup", "product_images"],
    "version": "1.0",
    "unique_name": "_books_v1_10_10",
}]


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

results = {} # {data_context: [{strategy_name, accuracy, precision, recall, f1_score, are_roc}]}

for product_data in product_datas:
    pprint.pprint(product_data)
    print("looking at", "../" + product_data["product_filepath"])

    productdf =  pd.read_csv("../" + product_data["product_filepath"])
    transactiondf = pd.read_csv("../" + product_data["transactions_filepath"])
    
    training_df_arr = []
    
    
    # join transactions by same user_id. into a dict of user_id: [transactions]
    user_transactions = {}
    for row in transactiondf[:100].iterrows():
        training_df_arr.append(row[1])
        user_id = row[1]["user_id"]
        if user_id not in user_transactions:
            user_transactions[user_id] = []
        user_transactions[user_id].append(row[1]['id'])
    
    # create df from transactionsdf
    transactiondf = pd.DataFrame(training_df_arr)
    
    print("user_transactions", len(user_transactions))
    print("transactiondf", transactiondf.shape)
    
    print(user_transactions.values())
    
    # split trainning and testing transactions
    past_transactions, test_transactions = train_test_split(list(user_transactions.values()), test_size=.2, random_state=42)
    # Display the shapes of the resulting DataFrames
    print("Training set shape:", len(past_transactions))
    print("Testing set shape:", len(test_transactions))
    
    # for each engine rec. Train, test:
    for rec_engine_class in engines_list[:1]:
        rec_engine:RecommendationAbstract  = rec_engine_class(products=productdf, product_data=product_data, transactions = transactiondf)
        rec_engine.train(transactiondf, auto_save=False)
        print(rec_engine.sim_score)
        
        for user_transactions in test_transactions:
            # Start test setup.
            # split in 3/4 if more than 4 transactions, or 1/2 if less than 4 transactions
            past_transactions, pred_transactions = train_test_split(user_transactions, test_size=.25, random_state=42)
            print('========= ', len(user_transactions), '=========', len(past_transactions), len(pred_transactions))
            print('attempting past_transactions', past_transactions)
            recs = rec_engine.recommend_from_past(past_transactions)
            # pprint.pprint(recs)
            
        
    
    
    


{'data_context': 'books',
 'features': ['product_title',
              'product_image',
              'product_soup',
              'product_images'],
 'product_filepath': 'data/products_books_v1_10_10.csv',
 'transactions_filepath': 'data/transactions_books_v1_10_10.csv',
 'unique_name': '_books_v1_10_10',
 'version': '1.0'}
looking at ../data/products_books_v1_10_10.csv
user_transactions 6
transactiondf (100, 4)
dict_values([['e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cbad6-c5a7-49b1-9eb2-558a648998d6'], ['e84cbad6-c5a7-49b1-9eb2-558a648998d6', 'e84cb

IndexError: index 0 is out of bounds for axis 0 with size 0